# Topic Modelling of used functions
## Function that collects all the calls

In [16]:
import ast

class CallCollector(ast.NodeVisitor):
    def __init__(self):
        self.calls = []

    def visit_Call(self, node):
        self.calls.append(node.func.attr)
    
    def get_calls(self):
        return(self.calls) 

## Test of the function

In [15]:
tree = ast.parse('''
connection = pymysql.connect()
with connection.cursor() as cursor:
    sql = ""
    cursor.execute(sql)
    test = cursor.fetchall()
    test = pd.DataFrame(test)
''')
cc = CallCollector()
cc.visit(tree)
cc.get_calls()

['connect', 'cursor', 'execute', 'fetchall', 'DataFrame']

## Get all the code blocks from a notebook

In [17]:
from nbformat import reads, NO_CONVERT
def code_extractor(jpt):
    notebook = reads(jpt, NO_CONVERT)
    cells = notebook.cells
    code_cells = [c for c in cells if c.cell_type == 'code']
    code = []
    for cell in code_cells:
        source = cell.source
        code.append(source)
    return code

## Extract all used functions 

In [31]:
import csv
# incresing the csv field size
import ctypes
csv.field_size_limit(int(ctypes.c_ulong(-1).value // 2))

calls = list()
with open('../data/jupyter_files.csv',"r", encoding="utf8") as csvfile:
    data = csv.DictReader(csvfile)
    for row in data:
        jpt = row['content']
        try:
            code = code_extractor(jpt)
            for c in code:
                tree = ast.parse(c)
                cc = CallCollector()
                cc.visit(tree)
                call = cc.get_calls()
                calls.append(call)
        except KeyboardInterrupt:
            break
        except:
            continue

Wall time: 0 ns


### Write the list to disk

In [32]:
import pickle

with open('../data/calls', 'wb') as fp:
    pickle.dump(calls, fp)

### Read the list from disk

In [3]:
import pickle

with open('../data/calls', 'rb') as fp:
    calls = pickle.load(fp)

## Inspect the data and clean up

In [4]:
len(calls)

30214

### Empty Blocks

In [5]:
calls.count([])

15802

In [6]:
#Function to remove all occurences of a specific item
def remove_values_from_list(the_list, val):
    return [value for value in the_list if value != val]
#remove all empty blocks
new_calls = remove_values_from_list(calls, [])
len(new_calls)

14412

## How many function calls are there per block?

In [7]:
import pandas as pd
c = []
for i in new_calls:
    c.append(len(i))
c = pd.Series(c)
c.describe()

count    14412.000000
mean         1.931030
std          2.260795
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max         65.000000
dtype: float64

In [10]:
len(c[c > 1])

5161

In [12]:
c[c > 1].describe()

count    5161.000000
mean        3.599884
std         3.151988
min         2.000000
25%         2.000000
50%         3.000000
75%         4.000000
max        65.000000
dtype: float64

# Topic Modelling of the functions with code blocks as a document

In [47]:
#Import all necessary libraries
import glob
import pandas as pd
import numpy as np

from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
warnings.filterwarnings('ignore')
import _pickle as pickle

import re
import random
import collections

from gensim import corpora, models
import gensim

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from scipy import stats

pyLDAvis.enable_notebook()
random.seed(1234)

from sklearn.metrics.cluster import adjusted_rand_score
pd.options.display.max_columns = None

c:\users\patrick\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [93]:
%%time
#build gensim corpus
texts = new_calls
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=3, no_above=0.4)
dictionary.compactify()
corpus = [dictionary.doc2bow(text) for text in texts]

Wall time: 279 ms


In [94]:
len(dictionary)

1176

In [103]:
%%time
ldamodelnormal = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=10, chunksize=100, update_every=0, alpha=1/15, random_state=1)

Wall time: 32.6 s


In [63]:
# helper to create descriptive tables (doc-topic probabilities) and visualizations for LDA models 
def getModelResults(ldamodel, corpus, dictionary):
    vis = pyLDAvis.gensim.prepare(ldamodel,corpus, dictionary, sort_topics=False)
    transformed = ldamodel.get_document_topics(corpus)
    df = pd.DataFrame.from_records([{v:k for v, k in row} for row in transformed])
    return vis, df    

In [54]:
# get the top topic per document into a list
def maxTop(x):
    mx = max(x,key=lambda item:item[1])
    if (mx[1]>0.0):
        return(mx[0])
    else:
        return 99

In [104]:
%%time
# get descriptive stuff for all models
normalv, dfnormal = getModelResults(ldamodelnormal, corpus, dictionary)

Wall time: 10.8 s


In [105]:
#print regular topics (top word probabilities)
ldamodelnormal.print_topics(num_words=8)

[(0,
  '0.075*"plot" + 0.063*"show" + 0.033*"ylabel" + 0.032*"xlabel" + 0.023*"title" + 0.020*"figure" + 0.020*"Series" + 0.018*"imshow"'),
 (1,
  '0.071*"apply" + 0.036*"execute" + 0.023*"basicConfig" + 0.019*"read_excel" + 0.018*"join" + 0.016*"close" + 0.015*"makedirs" + 0.015*"cursor"'),
 (2,
  '0.052*"count" + 0.048*"insert" + 0.046*"arange" + 0.033*"mean" + 0.031*"sqrt" + 0.029*"sin" + 0.022*"cos" + 0.021*"reset_index"'),
 (3,
  '0.240*"read_csv" + 0.061*"add" + 0.039*"linspace" + 0.033*"opts" + 0.032*"plot" + 0.026*"chdir" + 0.021*"json" + 0.016*"Simulation"'),
 (4,
  '0.078*"extension" + 0.063*"load" + 0.028*"append" + 0.028*"sum" + 0.022*"enable_eager_execution" + 0.020*"set_index" + 0.020*"to_csv" + 0.020*"exp"'),
 (5,
  '0.131*"array" + 0.071*"DataFrame" + 0.033*"Variable" + 0.033*"astype" + 0.027*"placeholder" + 0.026*"read_csv" + 0.021*"unique" + 0.018*"set_verbosity"'),
 (6,
  '0.075*"reshape" + 0.056*"split" + 0.041*"load_data" + 0.041*"describe" + 0.032*"show" + 0.022*"

In [106]:
# inspect regular topics - ATTENTION: ALL TOPICS ARE SHIFTED WITH ID +1 w.r.t. GENSIM
normalv

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.186533  0.136416       1        1  14.816159
1     -0.056016  0.092034       2        1   8.012075
2      0.164128 -0.081343       3        1   9.724637
3     -0.125676 -0.190303       4        1  11.707397
4     -0.050835  0.032871       5        1   8.715095
5      0.089987 -0.091781       6        1  10.938859
6      0.016356  0.104082       7        1   7.389490
7      0.185373  0.070376       8        1   9.716751
8     -0.086826 -0.137639       9        1  11.206043
9      0.050042  0.065286      10        1   7.773498, topic_info=     Category         Freq                 Term        Total  loglift  logprob
term                                                                          
3     Default  1028.000000             read_csv  1028.000000  30.0000  30.0000
27    Default   683.000000                 head   683.000000  29.0000  29.0000
2     Default   415.000000            load_data   415.000000  28.0000  28.0000
43    Default   473.000000                array   473.000000  27.0000  27.0000
24    Default   599.000000               append   599.000000  26.0000  26.0000
34    Default   264.000000            extension   264.000000  25.0000  25.0000
139   Default   159.000000              reshape   159.000000  24.0000  24.0000
4     Default   434.000000                 plot   434.000000  23.0000  23.0000
58    Default   171.000000                apply   171.000000  22.0000  22.0000
6     Default   169.000000                 load   169.000000  21.0000  21.0000
32    Default   337.000000            DataFrame   337.000000  20.0000  20.0000
20    Default   135.000000                count   135.000000  19.0000  19.0000
17    Default   221.000000                  get   221.000000  18.0000  18.0000
9     Default   256.000000                  add   256.000000  17.0000  17.0000
37    Default   172.000000                split   172.000000  16.0000  16.0000
124   Default   100.000000                 init   100.000000  15.0000  15.0000
64    Default   440.000000                 show   440.000000  14.0000  14.0000
15    Default   140.000000               insert   140.000000  13.0000  13.0000
127   Default   282.000000                 join   282.000000  12.0000  12.0000
28    Default   164.000000               arange   164.000000  11.0000  11.0000
163   Default   101.000000                  set   101.000000  10.0000  10.0000
340   Default   125.000000             describe   125.000000   9.0000   9.0000
68    Default    97.000000               format    97.000000   8.0000   8.0000
382   Default   110.000000             Variable   110.000000   7.0000   7.0000
33    Default   109.000000              execute   109.000000   6.0000   6.0000
174   Default   148.000000               ylabel   148.000000   5.0000   5.0000
60    Default   102.000000                 mean   102.000000   4.0000   4.0000
272   Default   142.000000               astype   142.000000   3.0000   3.0000
148   Default   127.000000          placeholder   127.000000   2.0000   2.0000
173   Default   145.000000               xlabel   145.000000   1.0000   1.0000
...       ...          ...                  ...          ...      ...      ...
553   Topic10    10.936776            set_style    14.995390   2.2388  -5.1523
301   Topic10    18.292860                lower    25.132860   2.2368  -4.6379
571   Topic10     8.070298  background_gradient    11.210914   2.2258  -5.4562
540   Topic10     6.476995   from_tensor_slices     9.032300   2.2219  -5.6762
475   Topic10    27.965254             download    39.119305   2.2188  -4.2135
59    Topic10    37.512054                  max    53.426903   2.2008  -3.9198
646   Topic10    41.171963              compile    63.005505   2.1290  -3.8267
504   Topic10    27.328119                  sub    41.647530   2.1331  -4.2365
208   Topic10    53.707829       read_data_sets    86.335121   2.0798 